<a href="https://colab.research.google.com/github/denncraft/AI_ML_notes/blob/main/Deep_Learning_2023_chapter02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import tensorflow as tf

In [119]:
class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    # создается матрица коэффициентов W с формой (input_size, output_size) и инициализируется случайными значениями
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)
    # создается вектор b с формой (output_size) и инициализируется нулями
    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  # создается метод запуска прямого прохода
  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  # создается вспомогательный метод для получения весов из слоя
  @property
  def weights(self):
    return [self.W, self.b]

# класс объединяющий слои в цепочку. он берет список слоев и реализует
# метод __call__ который просто вызывает по порядку слои в этом списке,
# передавая входные данные. он также имеет свойство weights, упрощающее
# наблюдение за весами.
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [120]:
# с помощью классов NaiveDense и NaiveSequential создется имитация модели Keras
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [121]:
import math
# генератор пакетов для итерации по пакетам и изображениям из набора MNIST
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

In [122]:
# этап обучения, нужно
# 1. вычислить прогнозы для изображений в пакете
# 2. найти значения потерь для эьтх прогнозов с учетом фактических меток
# 3. Вычислить градиент потерь с учетом весов модели
# 4. Скорректировать веса на небольшую величину в напрвлении противоположном градиенту

def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    # выполняется прямой проход с предсказанием
    predictions = model(images_batch)
    # вычисляется loss для текущего пакета изображений на основании полученный предсказаний
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
        labels_batch, predictions)
    # вычисляется средняя величины ошибки
    average_loss = tf.reduce_mean(per_sample_losses)
    # по среднему loss вычисляется градиент потерь -> список в котором каждое
    # значение соответствует весу параметра из model.weights
  gradients = tape.gradient(average_loss, model.weights)
  # корректировка занчений весов по полученному градиенту потерь
  update_weights(gradients, model.weights)
  # возврат средней ошики для информации
  return average_loss

In [129]:
# константа определяющая скорость обучений
learning_rate = 1e-3 # 1*10^-3 = 1*1/10^3 = 1*0,001 = 0,001
# функция обновления весов
# def update_weights(gradients, weights):
#   for g, w in zip(gradients, weights):
#     w.assign_sub(g * learning_rate)

# в Keras уже есть специальные оптимизаторы, которые делают тоже самое
from tensorflow import keras

optimizer = keras.optimizers.SGD(learning_rate)

def update_weights(gradients, weights):
  optimizer.apply_gradients(zip(gradients, weights))

In [124]:
# функция обучения, выполняющая цикл из множества повторений one_training_tstep
def fit(model, images, labels, epochs, batch_size=128):
  # вывод на экран номер исполняемой эпохи
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    # создается генератор пакетов на основе всего датасета
    batch_generator = BatchGenerator(images, labels, batch_size)
    # цикл пока генератор пакетов отдает пакеты
    for batch_counter in range(batch_generator.num_batches):
      # получает пакет изображений и соответствующих меток
      images_batch, labels_batch = batch_generator.next()
      # получает ошибку после шага обучения на текущем пакете изображений
      # при этом в ходе обучения выполняется расчет градиент и обновление коэффициентов
      loss = one_training_step(model, images_batch, labels_batch)
      # вывод на экран результата
      if batch_counter % 100 == 0:
        print(f"loss at batch {batch_counter}: {loss:.2f}")

In [134]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [ ]:
fit(model, train_images, train_labels, epochs=20, batch_size=250)

In [141]:
import numpy as np
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.84
